In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
from pprint import pprint
import json
from tqdm.auto import tqdm
profiles = json.load(open("profiles.json"))
print(len(profiles))
# De-duplicate profiles
profile_dict = {}
for p in profiles:
    if "profile_id" in p:
        profile_dict[p["profile_id"]] = p
print(len(profile_dict))

7791
7616


In [2]:
pprint(profiles[0]["experience"])

[{'company': {'employeeCountRange': {'end': 500, 'start': 201},
              'industries': ['Health, Wellness and Fitness']},
  'companyLogoUrl': 'https://media-exp1.licdn.com/dms/image/C4D0BAQHe-V0An_SsVA/company-logo_',
  'companyName': 'Diabetes UK',
  'companyUrn': 'urn:li:fs_miniCompany:483655',
  'entityUrn': 'urn:li:fs_position:(ACoAABYLkwwBZlmZPuUAK9ZUaloP8tnTfb1tUeo,1032229291)',
  'geoLocationName': 'London, United Kingdom',
  'geoUrn': 'urn:li:fs_geo:90009496',
  'locationName': 'London, United Kingdom',
  'region': 'urn:li:fs_region:(gb,4573)',
  'timePeriod': {'endDate': {'month': 9, 'year': 2019},
                 'startDate': {'month': 5, 'year': 2017}},
  'title': 'Insight Analyst'},
 {'company': {'employeeCountRange': {'end': 5000, 'start': 1001},
              'industries': ['Management Consulting']},
  'companyLogoUrl': 'https://media-exp1.licdn.com/dms/image/C4D0BAQGkhUlLEL8LTw/company-logo_',
  'companyName': 'PwC New Zealand',
  'companyUrn': 'urn:li:fs_miniCompa

In [3]:
edges = {}
for r in tqdm(profile_dict.values()):
    industries = []
    for i, e in enumerate(r["experience"]):
        if i < (len(r["experience"]) - 1):
            try:
                year = e["timePeriod"]["startDate"]["year"]
                assert year >= 2010 and year <= 2019
                industry = e["company"]["industries"][0]
                prevIndustry = r["experience"][i + 1]["company"]["industries"][0]
            except:
                continue
            edge_id = prevIndustry + "_" + industry
            if edge_id not in edges:
                edges[edge_id] = {
                    "source": prevIndustry,
                    "target": industry,
                    "val": 0
                }
            edges[edge_id]["val"] += 1

  0%|          | 0/7616 [00:00<?, ?it/s]

In [4]:
df = pd.DataFrame(edges.values()).sort_values(by="val", ascending=False)
df

,source,target,val
4,Management Consulting,Management Consulting,1797
24,Accounting,Accounting,975
74,Accounting,Management Consulting,412
6,Information Technology and Services,Information Technology and Services,372
34,Banking,Banking,296
...,...,...,...
1159,Luxury Goods & Jewelry,Management Consulting,1
1157,Food & Beverages,Sporting Goods,1
1156,Sporting Goods,Marketing and Advertising,1
1153,Information Technology and Services,Consumer Goods,1


In [5]:
%%time
with pd.ExcelWriter('linkedin.xlsx', mode='a', if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name='Edges (Industries)', index=False)

CPU times: user 4.49 s, sys: 88.7 ms, total: 4.58 s
Wall time: 4.58 s


In [6]:
# Industries of interest
ioi = ["Management Consulting", "Government Administration"]
df[df.source.isin(ioi) & df.target.isin(ioi)]

,source,target,val
4,Management Consulting,Management Consulting,1797
106,Government Administration,Government Administration,148
41,Government Administration,Management Consulting,107
61,Management Consulting,Government Administration,67


In [7]:
profiles_of_interest = []
for r in tqdm(profile_dict.values()):
    industries = []
    for i, e in enumerate(r["experience"]):
        if i < (len(r["experience"]) - 1):
            try:
                year = e["timePeriod"]["startDate"]["year"]
                assert year >= 2010 and year <= 2019
                industry = e["company"]["industries"][0]
                prevIndustry = r["experience"][i + 1]["company"]["industries"][0]
                companyName = e["companyName"]
                prevCompanyName = r["experience"][i + 1]["companyName"]
            except:
                continue
            if prevIndustry != industry and prevIndustry in ioi and industry in ioi:
                r["source_company"] = prevCompanyName
                r["target_company"] = companyName
                r["source_industry"] = prevIndustry
                r["target_industry"] = industry
                profiles_of_interest.append(r.copy())

  0%|          | 0/7616 [00:00<?, ?it/s]

In [8]:
df = pd.DataFrame(profiles_of_interest)
df

,summary,industryName,lastName,locationName,student,geoCountryName,geoCountryUrn,geoLocationBackfilled,elt,industryUrn,firstName,entityUrn,geoLocation,geoLocationName,location,headline,displayPictureUrl,img_100_100,img_200_200,img_400_400,img_800_800,profile_id,profile_urn,member_urn,experience,education,languages,publications,certifications,volunteer,honors,source_company,target_company,source_industry,target_industry,birthDate,maidenName,$anti_abuse_metadata
0,I'm interested in the interaction between gove...,International Trade & Development,Cheng,New Zealand,False,New Zealand,urn:li:fs_geo:105490917,False,False,urn:li:fs_industry:141,David,urn:li:fs_profile:ACoAABODMNIBNvO-EzoLRsjP52v6...,{'geoUrn': 'urn:li:fs_geo:90010304'},Wellington Region,{'basicLocation': {'countryCode': 'nz'}},Economic Policy Analyst at British High Commis...,https://media-exp1.licdn.com/dms/image/C5603AQ...,100_100/0/1630298269234?e=1643241600&v=beta&t=...,200_200/0/1630298269234?e=1643241600&v=beta&t=...,400_400/0/1630298269234?e=1643241600&v=beta&t=...,800_800/0/1630298269234?e=1643241600&v=beta&t=...,ACoAABODMNIBNvO-EzoLRsjP52v6dts-fTBc_Uw,urn:li:fs_miniProfile:ACoAABODMNIBNvO-EzoLRsjP...,urn:li:member:327364818,"[{'locationName': 'British High Commission, We...",[{'entityUrn': 'urn:li:fs_education:(ACoAABODM...,"[{'name': 'English', 'proficiency': 'NATIVE_OR...",[],[],[],[{'description': 'Awarded to support my overse...,The Treasury - New Zealand,PwC New Zealand,Government Administration,Management Consulting,NaN,NaN,NaN
1,NaN,Legal Services,Colombié,New Zealand,False,New Zealand,urn:li:fs_geo:105490917,False,False,urn:li:fs_industry:10,Vincent,urn:li:fs_profile:ACoAAAYwaSIB4NlAAzM0m_SvLP7P...,{'geoUrn': 'urn:li:fs_geo:100749476'},"Auckland, Auckland",{'basicLocation': {'countryCode': 'nz'}},"Manager, Supervision at Financial Markets Auth...",https://media-exp1.licdn.com/dms/image/C4D03AQ...,100_100/0/1517613985416?e=1643241600&v=beta&t=...,200_200/0/1517613985416?e=1643241600&v=beta&t=...,400_400/0/1517613985416?e=1643241600&v=beta&t=...,800_800/0/1517613985416?e=1643241600&v=beta&t=...,ACoAAAYwaSIB4NlAAzM0m_SvLP7P-QuMeb6BD1Y,urn:li:fs_miniProfile:ACoAAAYwaSIB4NlAAzM0m_Sv...,urn:li:member:103835938,"[{'locationName': 'Auckland, New Zealand', 'en...",[{'entityUrn': 'urn:li:fs_education:(ACoAAAYwa...,"[{'name': 'Chinese Mandarin', 'proficiency': '...",[],[],"[{'role': 'Volunteering and Fundraising', 'com...",[],PwC New Zealand,Financial Markets Authority - New Zealand,Management Consulting,Government Administration,NaN,NaN,NaN
2,Experienced Software Engineer with a demonstra...,Computer Software,Hunter,New Zealand,False,New Zealand,urn:li:fs_geo:105490917,False,False,urn:li:fs_industry:4,Tracey,urn:li:fs_profile:ACoAAAaC5joBD4xDAgWN6f2O5HBD...,{'geoUrn': 'urn:li:fs_geo:100749476'},"Auckland, Auckland",{'basicLocation': {'countryCode': 'nz'}},Senior Consultant at First Software Ltd,https://media-exp1.licdn.com/dms/image/C5603AQ...,100_100/0/1516453387563?e=1643241600&v=beta&t=...,200_200/0/1516453387563?e=1643241600&v=beta&t=...,400_400/0/1516453387563?e=1643241600&v=beta&t=...,800_800/0/1516453387563?e=1643241600&v=beta&t=...,ACoAAAaC5joBD4xDAgWN6f2O5HBDIP_1P2j0N6g,urn:li:fs_miniProfile:ACoAAAaC5joBD4xDAgWN6f2O...,urn:li:member:109241914,"[{'locationName': 'Auckland, New Zealand', 'en...",[{'courses': ['urn:li:fs_course:(ACoAAAaC5joBD...,"[{'name': 'English', 'proficiency': 'NATIVE_OR...",[],"[{'authority': 'Xero', 'name': 'Xero Advisor C...",[],[],Auckland Council,PwC New Zealand,Government Administration,Management Consulting,NaN,NaN,NaN
3,NaN,Banking,Butterfield,NaN,False,New Zealand,urn:li:fs_geo:105490917,False,False,urn:li:fs_industry:41,Kathleen,urn:li:fs_profile:ACoAABdReZABZiW6bUr-9bPJB2p9...,{'geoUrn': 'urn:li:fs_geo:100749476'},"Auckland, Auckland",{'basicLocation': {'countryCode': 'nz'}},Financial Analyst at ASB Bank,NaN,NaN,NaN,NaN,NaN,ACoAABdReZABZiW6bUr-9bPJB2p9HbnBoSqeJQE,urn:li:fs_miniProfile:ACoAABdReZABZiW6bUr-9bPJ...,urn:li:me

In [9]:
df[df.duplicated(subset="member_urn", keep=False)]

,summary,industryName,lastName,locationName,student,geoCountryName,geoCountryUrn,geoLocationBackfilled,elt,industryUrn,firstName,entityUrn,geoLocation,geoLocationName,location,headline,displayPictureUrl,img_100_100,img_200_200,img_400_400,img_800_800,profile_id,profile_urn,member_urn,experience,education,languages,publications,certifications,volunteer,honors,source_company,target_company,source_industry,target_industry,birthDate,maidenName,$anti_abuse_metadata
4,NaN,Management Consulting,W.,New Zealand,False,New Zealand,urn:li:fs_geo:105490917,False,False,urn:li:fs_industry:11,Zach,urn:li:fs_profile:ACoAACD1vbMBgmz5WvTpybhxVC1b...,{'geoUrn': 'urn:li:fs_geo:100749476'},"Auckland, Auckland",{'basicLocation': {'countryCode': 'nz'}},Associate at Boston Consulting Group (BCG),https://media-exp1.licdn.com/dms/image/C5603AQ...,100_100/0/1634945141282?e=1643241600&v=beta&t=...,200_200/0/1634945141282?e=1643241600&v=beta&t=...,400_400/0/1634945141282?e=1643241600&v=beta&t=...,800_800/0/1634945141282?e=1643241600&v=beta&t=...,ACoAACD1vbMBgmz5WvTpybhxVC1bZ-dLtS-ezKA,urn:li:fs_miniProfile:ACoAACD1vbMBgmz5WvTpybhx...,urn:li:member:552975795,"[{'locationName': 'Sydney, New South Wales, Au...",[{'entityUrn': 'urn:li:fs_education:(ACoAACD1v...,[],[],[],"[{'role': 'National Finance Officer', 'company...",[{'description': 'Econ 321 - Advanced Economet...,PwC New Zealand,The Treasury - New Zealand,Management Consulting,Government Administration,NaN,NaN,NaN
5,NaN,Management Consulting,W.,New Zealand,False,New Zealand,urn:li:fs_geo:105490917,False,False,urn:li:fs_industry:11,Zach,urn:li:fs_profile:ACoAACD1vbMBgmz5WvTpybhxVC1b...,{'geoUrn': 'urn:li:fs_geo:100749476'},"Auckland, Auckland",{'basicLocation': {'countryCode': 'nz'}},Associate at Boston Consulting Group (BCG),https://media-exp1.licdn.com/dms/image/C5603AQ...,100_100/0/1634945141282?e=1643241600&v=beta&t=...,200_200/0/1634945141282?e=1643241600&v=beta&t=...,400_400/0/1634945141282?e=1643241600&v=beta&t=...,800_800/0/1634945141282?e=1643241600&v=beta&t=...,ACoAACD1vbMBgmz5WvTpybhxVC1bZ-dLtS-ezKA,urn:li:fs_miniProfile:ACoAACD1vbMBgmz5WvTpybhx...,urn:li:member:552975795,"[{'locationName': 'Sydney, New South Wales, Au...",[{'entityUrn': 'urn:li:fs_education:(ACoAACD1v...,[],[],[],"[{'role': 'National Finance Officer', 'company...",[{'description': 'Econ 321 - Advanced Economet...,"Ministry of Business, Innovation and Employment",PwC New Zealand,Government Administration,Management Consulting,NaN,NaN,NaN
11,Experienced Analyst with a demonstrated histor...,Government Administration,King,Canada,False,Canada,urn:li:fs_geo:101174742,False,False,urn:li:fs_industry:75,Laura,urn:li:fs_profile:ACoAABBhWtMBAH0W90gZWtnvKJMA...,{'geoUrn': 'urn:li:fs_geo:101174742'},NaN,{'basicLocation': {'countryCode': 'ca'}},Senior Analyst at The Treasury - New Zealand,https://media-exp1.licdn.com/dms/image/C4E03AQ...,100_100/0/1516638658327?e=1643241600&v=beta&t=...,200_200/0/1516638658327?e=1643241600&v=beta&t=...,400_400/0/1516638658327?e=1643241600&v=beta&t=...,800_800/0/1516638658327?e=1643241600&v=beta&t=...,ACoAABBhWtMBAH0W90gZWtnvKJMAQbqG9rz6cGQ,urn:li:fs_miniProfile:ACoAABBhWtMBAH0W90gZWtnv...,urn:li:member:274815699,"[{'locationName': 'Wellington, Wellington Regi...",[{'entityUrn': 'urn:li:fs_education:(ACoAABBhW...,[],"[{'date': {'year': 2016}, 'name': 'Social Capi...",[],[],[{'description': 'Awarded into the Golden Key ...,PwC New Zealand,Ministry of Social Development (MSD),Management Consulting,Government Administration,"{'month': 8, 'day': 28}",NaN,NaN
12,Experienced Analyst with a demonstrated histor...,Government Administration,King,Canada,False,Canada,urn:li:fs_geo:101174742,False,False,urn:li:fs_industry:75,Laura,urn:li:fs_profile:ACoAABBhWtMBAH0W90gZWtnvKJMA...,{'geoUrn': 'urn:li:fs_geo:101174742'},NaN,{'basicLocation': {'countryCode': 'ca'}},Senior Analyst at The Treasury - New Zealand,https://media-exp1.licdn.com/dms/image/C4E03AQ...,100_100/0/1516638658327?e=1643241600&v=beta&t=...,200_200/0/1516

In [10]:
df.source_company.value_counts()

PwC New Zealand                                            34
KPMG New Zealand                                           18
Auckland Council                                           14
Inland Revenue NZ                                          14
The Treasury - New Zealand                                  7
Deloitte New Zealand                                        6
Ministry of Business, Innovation and Employment             5
Auckland Transport                                          4
New Zealand Treasury                                        4
Ministry of Justice - New Zealand                           4
Land Information New Zealand                                3
Ministry for Primary Industries (MPI)                       3
Ministry of Health New Zealand                              3
Ministry of Social Development (MSD)                        3
Deloitte                                                    3
Department of Corrections- NZ                               2
State Se

In [11]:
people = json.load(open("people.json"))
len(people)

11574

In [12]:
lookup = {}
for p in people:
    lookup[p["urn_id"]] = p

In [13]:
df.iloc[0]

summary                  I'm interested in the interaction between gove...
industryName                             International Trade & Development
lastName                                                             Cheng
locationName                                                   New Zealand
student                                                              False
geoCountryName                                                 New Zealand
geoCountryUrn                                      urn:li:fs_geo:105490917
geoLocationBackfilled                                                False
elt                                                                  False
industryUrn                                         urn:li:fs_industry:141
firstName                                                            David
entityUrn                urn:li:fs_profile:ACoAABODMNIBNvO-EzoLRsjP52v6...
geoLocation                           {'geoUrn': 'urn:li:fs_geo:90010304'}
geoLocationName          

In [14]:
df["URL"] = f'=HYPERLINK("https://www.linkedin.com/in/' + df.profile_id + '")'

In [15]:
df.URL.iloc[0]

'=HYPERLINK("https://www.linkedin.com/in/ACoAABODMNIBNvO-EzoLRsjP52v6dts-fTBc_Uw")'

In [16]:
df.target_company.value_counts()

PwC New Zealand                                            43
KPMG New Zealand                                           33
Deloitte New Zealand                                       20
Auckland Council                                            9
Financial Markets Authority - New Zealand                   7
The Treasury - New Zealand                                  5
Ministry for Primary Industries (MPI)                       5
ACC New Zealand                                             5
Deloitte                                                    4
Inland Revenue NZ                                           4
Department of Conservation (DOC)                            3
Land Information New Zealand                                2
Auckland Tourism, Events & Economic Development (ATEED)     2
New Zealand Customs Service                                 2
Auckland Transport                                          2
Ministry of Business, Innovation and Employment             2
Oranga T

In [17]:
df.columns

Index(['summary', 'industryName', 'lastName', 'locationName', 'student',
       'geoCountryName', 'geoCountryUrn', 'geoLocationBackfilled', 'elt',
       'industryUrn', 'firstName', 'entityUrn', 'geoLocation',
       'geoLocationName', 'location', 'headline', 'displayPictureUrl',
       'img_100_100', 'img_200_200', 'img_400_400', 'img_800_800',
       'profile_id', 'profile_urn', 'member_urn', 'experience', 'education',
       'languages', 'publications', 'certifications', 'volunteer', 'honors',
       'source_company', 'target_company', 'source_industry',
       'target_industry', 'birthDate', 'maidenName', '$anti_abuse_metadata',
       'URL'],
      dtype='object')

In [18]:
df[['firstName',
 'lastName',
 'industryName',
 'summary',
 'locationName',
 'geoCountryName',
 'geoLocationName',
 'headline',
 'experience',
 'education',
 'languages',
 'publications',
 'certifications',
 'volunteer',
 'honors',
 'source_company',
 'target_company',
 'source_industry',
 'target_industry',
 'birthDate',
 'maidenName', "URL"]].to_excel("linkedin_profiles_of_interest.xlsx", index=False)